In [1]:
## Import modules
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
from demo import make_animation
from skimage import img_as_ubyte
from moviepy.editor import *
from demo import load_checkpoints
import warnings
warnings.filterwarnings("ignore")

In [2]:
## Load source and driving video
source_image = imageio.imread('C:\\Users\\Desktop\\DeepFake\\first_order_model\\img2.jpeg')
driving_video = imageio.mimread("C:\\Users\\Desktop\\DeepFake\\first_order_model\\Test1.mp4")

## Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

In [3]:
def display(source, driving,  generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    
HTML(display(source_image, driving_video).to_html5_video())

In [4]:
#Function to display only the video and driving video

def display2(generated):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
    ims = []
    for i in range(len(generated)):
        cols = [generated[i]]
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50)
    plt.close()
    return ani

In [5]:
#Loading Pre-trained First Order Model
from demo import load_checkpoints
generator2, kp_detector2 = load_checkpoints(config_path='C:\\Users\\kiran\\Desktop\\Laxman\\DeepFake\\first_order_model\\config\\vox-256.yaml', 
                            checkpoint_path='C:\\Users\\kiran\\Desktop\\Laxman\\DeepFake\\vox-adv-cpk.pth.tar')

In [6]:
predictions = make_animation(source_image, driving_video, generator2, kp_detector2, relative=True)

100%|████████████████████████████████████████████████████████████████████████████████| 206/206 [03:22<00:00,  1.02it/s]


In [7]:
HTML(display2(predictions).to_html5_video())

In [8]:
#Converting animation into video
videoclip = VideoFileClip("C:\\Users\\Desktop\\DeepFake\\first_order_model\\Test1.mp4")
vfp=videoclip.fps
imageio.mimsave('Generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=vfp)

In [9]:
#Extracting audio from original video
import os
import sys
from moviepy.editor import VideoFileClip


def convert_video_to_audio_moviepy(video_file, output_ext="mp3"):
    """Converts video to audio using MoviePy library
    that uses `ffmpeg` under the hood"""
    filename, ext = os.path.splitext(video_file)
    clip = VideoFileClip(video_file)
    clip.audio.write_audiofile(f"{filename}.{output_ext}")


vf="C:\\Users\\kiran\\Desktop\\Laxman\\DeepFake\\first_order_model\\Test1.mp4"
convert_video_to_audio_moviepy(vf)

MoviePy - Writing audio in C:\Users\Desktop\DeepFake\first_order_model\Test1.mp3


MoviePy - Done.


In [10]:
#Using ffmpeg method to combine video and audio

import ffmpeg

input_video = ffmpeg.input('Generated.mp4')

input_audio = ffmpeg.input('C:\\Users\\kiran\\Desktop\\Laxman\\DeepFake\\first_order_model\\Test1.mp3')

ffmpeg.concat(input_video, input_audio, v=1, a=1).output('Output.mp4').run()

(None, None)

In [11]:
video = VideoFileClip('Output.mp4')
video.ipython_display(width = 280, height = 320)

Moviepy - Building video __temp__.mp4.
MoviePy - Writing audio in __temp__TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
